**Tutorial pekan ke-3**, pemodelan vektor kata-konteks (term, context) dengan **PMI** (Pointwise Mutual Information)



Import library yang dibutuhkan

In [122]:
import nltk
import numpy as np
import re



Memproses teks masukan (terdiri atas beberapa kalimat), tokenisasi.

In [123]:
article_text = 'Saya sedang belajar Bahasa Inggris.\n'
article_text += 'Adik sedang belajar IPA.\n'
#article_text += 'Ibu menonton video rekaman pelajaran Bahasa Arab.\n'
#article_text += 'Jumlah angka penderita Covid-19 di Indonesia masih terus naik.\n'
#article_text += 'Jumlah tenaga kesehatan yang terinfeksi Covid-19 masih bertambah.\n'
#article_text += 'Masyarakat Indonesia harus disiplin supaya Covid-19 dapat terkendali.\n'


Hitung frekuensi kemunculan tiap token/kata

In [124]:
nltk.download('punkt')
corpus = nltk.sent_tokenize(article_text)

for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i]) # hapus punctuation / tanda baca
    corpus [i] = re.sub(r'\s+',' ',corpus [i]) # hapus spasi berlebih

vocab_freq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in vocab_freq:
          vocab_freq[token] = 1
        else:
          vocab_freq[token] += 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [125]:
print(vocab_freq)

{'saya': 1, 'sedang': 2, 'belajar': 2, 'bahasa': 1, 'inggris': 1, 'adik': 1, 'ipa': 1}


Buat matriks *co-occurrence*

In [126]:
# inisialisasi matriks co-occurrence
co_occurrence_mat = {}
for token_1 in vocab_freq.keys():
  co_occurrence_terms = []
  for token_2 in vocab_freq.keys():
     co_occurrence_mat[(token_1,token_2)] = 0

# set ukuran window
window_size = 2 # contoh, ukuran window = 2

# inisialisasi jumlah bigram (term, context) yang muncul
sum_term_context = 0 # jumlah kemunculan term, context

# proses
for sentence in corpus:
  tokens = nltk.word_tokenize(sentence)
  for i in range(0,len(tokens)):
    # konteks kata-kata sebelah kiri
    left_index = i-1
    while left_index >= 0 and left_index >= i-window_size:
      token_1 = tokens[i]
      token_2 = tokens[left_index]
      co_occurrence_mat[(token_1,token_2)] += 1
      sum_term_context += 1
      left_index = left_index - 1
    # konteks kata-kata sebelah kanan
    right_index = i+1
    while right_index < len(tokens) and right_index <= i+window_size:
      token_1 = tokens[i]
      token_2 = tokens[right_index] 
      co_occurrence_mat[(token_1,token_2)] += 1    
      sum_term_context += 1
      right_index = right_index + 1
 

     



Inisialisasi jumlah kemunculan sebuah term sebagai bagian dari bigram

In [127]:
bigram_count = {}

Buat fungsi untuk print matriks co-occurrence, untuk contoh data yang tidak terlalu banyak. Proses ini sekaligus mengisi nilai jumlah kemunculan sebuah term sebagai bagian dari bigram.

In [128]:
def print_cooccurrence_mat():
  str_token = '\t'
  for token in vocab_freq.keys():
    str_token += '\t\t'+token
  print(str_token)
  for token_1 in vocab_freq.keys():
    str_row = token_1+'\t'
    curr_bigram_count = 0
    bigram_count[token_1] = 0
    #print(token_1)
    for token_2 in vocab_freq.keys():
      str_row += '\t\t'+str(co_occurrence_mat[(token_1,token_2)])
      curr_bigram_count += co_occurrence_mat[(token_1,token_2)] # update jumlah kemunculan term
    bigram_count[token_1] = curr_bigram_count # assignemnt jumlah kemunculan term sebagai bagian dari bigram
    print(str_row)


In [129]:
print_cooccurrence_mat()

			saya		sedang		belajar		bahasa		inggris		adik		ipa
saya			0		1		1		0		0		0		0
sedang			1		0		2		1		0		1		1
belajar			1		2		0		1		1		1		1
bahasa			0		1		1		0		1		0		0
inggris			0		0		1		1		0		0		0
adik			0		1		1		0		0		0		0
ipa			0		1		1		0		0		0		0


In [130]:
print(bigram_count)

{'saya': 2, 'sedang': 6, 'belajar': 7, 'bahasa': 3, 'inggris': 2, 'adik': 2, 'ipa': 2}


Print isi matriks co-occurrence, dan print jumlah term/context yang muncul pada dokumen

In [131]:
print(co_occurrence_mat)
print(sum_term_context)

{('saya', 'saya'): 0, ('saya', 'sedang'): 1, ('saya', 'belajar'): 1, ('saya', 'bahasa'): 0, ('saya', 'inggris'): 0, ('saya', 'adik'): 0, ('saya', 'ipa'): 0, ('sedang', 'saya'): 1, ('sedang', 'sedang'): 0, ('sedang', 'belajar'): 2, ('sedang', 'bahasa'): 1, ('sedang', 'inggris'): 0, ('sedang', 'adik'): 1, ('sedang', 'ipa'): 1, ('belajar', 'saya'): 1, ('belajar', 'sedang'): 2, ('belajar', 'belajar'): 0, ('belajar', 'bahasa'): 1, ('belajar', 'inggris'): 1, ('belajar', 'adik'): 1, ('belajar', 'ipa'): 1, ('bahasa', 'saya'): 0, ('bahasa', 'sedang'): 1, ('bahasa', 'belajar'): 1, ('bahasa', 'bahasa'): 0, ('bahasa', 'inggris'): 1, ('bahasa', 'adik'): 0, ('bahasa', 'ipa'): 0, ('inggris', 'saya'): 0, ('inggris', 'sedang'): 0, ('inggris', 'belajar'): 1, ('inggris', 'bahasa'): 1, ('inggris', 'inggris'): 0, ('inggris', 'adik'): 0, ('inggris', 'ipa'): 0, ('adik', 'saya'): 0, ('adik', 'sedang'): 1, ('adik', 'belajar'): 1, ('adik', 'bahasa'): 0, ('adik', 'inggris'): 0, ('adik', 'adik'): 0, ('adik', 'ipa

Buat matriks probability(term, context)

In [132]:
# definisikan dan isi matriks probability (term, context)
term_context_prob = {}
for token_1 in vocab_freq.keys():
  for token_2 in vocab_freq.keys():    
    term_context_prob[(token_1,token_2)] = co_occurrence_mat[(token_1,token_2)]/sum_term_context



Buat fungsi print matriks probability (term, context), untuk contoh data sedikit, supaya lebih mudah diperiksa isinya.

In [133]:
def print_term_context_prob():
  str_token = '\t'
  for token in vocab_freq.keys():
    str_token += '\t\t'+token
  print(str_token)
  for token_1 in vocab_freq.keys():
    str_row = token_1+'\t'
    for token_2 in vocab_freq.keys():
      str_row += '\t\t'+str(round(term_context_prob[(token_1,token_2)],4))
    print(str_row)


In [134]:
print_term_context_prob()

			saya		sedang		belajar		bahasa		inggris		adik		ipa
saya			0.0		0.0417		0.0417		0.0		0.0		0.0		0.0
sedang			0.0417		0.0		0.0833		0.0417		0.0		0.0417		0.0417
belajar			0.0417		0.0833		0.0		0.0417		0.0417		0.0417		0.0417
bahasa			0.0		0.0417		0.0417		0.0		0.0417		0.0		0.0
inggris			0.0		0.0		0.0417		0.0417		0.0		0.0		0.0
adik			0.0		0.0417		0.0417		0.0		0.0		0.0		0.0
ipa			0.0		0.0417		0.0417		0.0		0.0		0.0		0.0


Buat matriks PMI (term, context)

In [135]:
# definisikan dan isi matriks probability (term, context)
pmi_mat = {}
# import lib math untuk menghitung log2
import math

for token_1 in vocab_freq.keys():
  for token_2 in vocab_freq.keys(): 
    token_1_prob = bigram_count[token_1]/sum_term_context
    token_2_prob = bigram_count[token_2]/sum_term_context
    if term_context_prob[(token_1,token_2)] > 0:
      pmi_mat[(token_1,token_2)] = round(math.log2(term_context_prob[(token_1,token_2)]/(token_1_prob*token_2_prob)), 4)
    else: # kalau nilai probability = 0, tidak bisa dihitung log 2 -nya
      pmi_mat[(token_1,token_2)] = None

Print matriks PMI

In [136]:
def print_pmi_mat():
  str_token = '\t'
  for token in vocab_freq.keys():
    str_token += '\t\t'+token
  print(str_token)
  for token_1 in vocab_freq.keys():
    str_row = token_1+'\t'
    #print(token_1)
    for token_2 in vocab_freq.keys():
      str_row += '\t\t'+str(pmi_mat[(token_1,token_2)])
    print(str_row)

In [137]:
print_pmi_mat()

			saya		sedang		belajar		bahasa		inggris		adik		ipa
saya			None		1.0		0.7776		None		None		None		None
sedang			1.0		None		0.1926		0.415		None		1.0		1.0
belajar			0.7776		0.1926		None		0.1926		0.7776		0.7776		0.7776
bahasa			None		0.415		0.1926		None		2.0		None		None
inggris			None		None		0.7776		2.0		None		None		None
adik			None		1.0		0.7776		None		None		None		None
ipa			None		1.0		0.7776		None		None		None		None
